# Extracting Data From Brokerage Notes Using Python

The brokerage note is a document generated by the broker when you negotiate in the stock market. Normally, it is generated every day you make a transaction and it is sent to you (or included in your account in the Broker portal) in a PDF format. These notes include all the information on the items you negotiated, such as price, quantity and fees. These are important information to analyze and control your portfolio and also to declare your taxes.

When you have only a few notes it is an easy task to copy the information you need and paste it into a worksheet. However, when you operate many days, it can be a time-consuming task to copy and paste from all the notes. Besides my interest in automation and for developing new skills, this was one of my biggest motivation to work on this project.

The objective of this project is to get the necessary information from a brokerage note in PDF format and save it to a worksheet, where it can be easier handled. I used a brokerage note from Clear (a Brazilian broker) with three operations of stock negotiation.

Here is a summary of the information we will get from the PDF file:
- Date of transaction;
- Buy/Sell operation for each transaction;
- Stock title;
- Price of each unit of stock;
- Quantity of stocks negociated.

## Installing the libraries

In [ ]:
!pip install pandas
!pip install tabula-py
!pip install regex

## Reading the PDF file using Tabula

In [2]:
#import libraries
import pandas as pd
import tabula

In [30]:
#storing the whole path of the PDF file.
filename = "/Users/hugo/Documents/brokerage_note.pdf"

In [ ]:
#reading PDF file
data = tabula.read_pdf(filename, multiple_tables=False, pages='all', stream=True, guess=False)

In [28]:
#creating data frame
df = data[0]

**Visualizing the data frame**

Obs.: I filtered out some rows to preserve sensitive information.

In [38]:
df

,Unnamed: 0,NOTA DE CORRETAGEM,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,Nr. nota Folha,Data pregão,NaN
1,NaN,NaN,929897 1,NaN,08/03/2019
2,NaN,CLEAR CORRETORA - GRUPO XP,NaN,NaN,NaN
16,Negócios realizados,NaN,NaN,NaN,NaN
17,Q Negociação C/V,Tipo mercado Prazo Especificação do título Obs...,Valor Operação / Ajuste,NaN,D/C
18,1-BOVESPA V,FRACIONARIO ENGIE BRASILON NM 5,"40,00",NaN,"200,00 C"
19,1-BOVESPA C,FRACIONARIO GERDAUON ED N1 10,"11,63",NaN,"116,30 D"
20,1-BOVESPA C,VISTA PARANAPANEMAON NM 100,"1,47",NaN,"147,00 D"
21,Resumo dos Negócios,Resumo Financeiro,NaN,NaN,NaN
22,Debêntures,"0,00 Clearing",NaN,NaN,NaN


## Getting the right information using Regex

In [14]:
#import libraries
import re

**Date:**

The first information we are getting is the date of operation. We won't need to use Regex in this case because the date is the only information in the cell of column "Unnamed: 4" and row 1. After filtering it, we can apply the strip() function to clean any possible empty characters before or after the date.

In [15]:
date = df['Unnamed: 4'].iloc[1].strip()

**Getting Data from Operations**

For the information on each operation, we will not use the row number as a reference because we will not know how many operations we have in each note. We will need another way to reference the operation rows. It is possible to see that all the rows we need has the value "1-BOVESPA " in column "Unnamed: 0".

**- Buy/Sell Operation:** it is defined in the column "Unnamed: 0". It is the last information in the cell, being "V" for sell (venda in Portuguese) and "C" for buy (compra in Portuguese). Our pattern can be a single letter at the end of the string, having one empty character before and none or some empty character at the end.

**- Stock Title:** it is inside the column "NOTA DE CORRETAGEM". It is between the value "VISTA" (when buying in a pack of 100 stocks) or "FRACIONARIO" (when buying individually) and the quantity. Thus, to get the stock title value, we can first check if the cell contains "FRACIONARIO" or "VISTA" and use this info to define what is before our target. Also, we can define that we will have a number (of one or more digits) after our target.

**- Price:** it will be easy to get because as the date it is the only value in the cell. Thus, we can simply filter the right index and get the value in the column "Unnamed: 2""

**- Quantity:** it is inside the same cell as the stock title. It is the last value in the cell, so we can get it using a similar expression as the buy/sell operation, but looking for a number (with one or more digits) instead of a letter.

**Putting it all together**

We can create a list with all the data from each row and store it in a variable called "row_data". We will also add our date to this list since it is the same date for all the operations. Then, we are going to append this list to another list that will have the information from all rows. We are going to call this list "note_data".

In [16]:
note_data = []

operations = list(df[df['Unnamed: 0'].str.contains("1-BOVESPA",na=False)].index)

for current_row in operations:
    
    #buy/sell operation
    cell_value = df['Unnamed: 0'].iloc[current_row]
    c_vRegex = re.compile(r'(?<=\s)[a-zA-Z]\s*$')
    c_v = c_vRegex.search(cell_value).group(0).strip()
    
    #stock title
    cell_value = df['NOTA DE CORRETAGEM'].iloc[current_row]
    if 'FRACIONARIO' in cell_value:
        stock_titleRegex = re.compile(r'(?<=FRACIONARIO\s)(.*)(?=\s\d+)')
    elif 'VISTA' in cell_value:
        stock_titleRegex = re.compile(r'(?<=VISTA\s)(.*)(?=\s\d+)')
    stock_title = stock_titleRegex.search(cell_value).group(0).strip()
    
    #price
    price = df['Unnamed: 2'].iloc[current_row].strip()
    
    #quantity
    cell_value = df['NOTA DE CORRETAGEM'].iloc[current_row]
    quantityRegex = re.compile(r'(?<=\s)\d*\s*$')
    quantity = quantityRegex.search(cell_value).group(0).strip()
    
    row_data = [date, c_v, stock_title, price, quantity]
    note_data.append(row_data)

In [17]:
note_data

[['08/03/2019', 'V', 'ENGIE BRASILON NM', '40,00', '5'],
 ['08/03/2019', 'C', 'GERDAUON ED N1', '11,63', '10'],
 ['08/03/2019', 'C', 'PARANAPANEMAON NM', '1,47', '100']]

## Exporting to Excel

To export our data to an Excel file we are going to use the Pandas library. The first step is to create a data frame from our "note_date" list. We need to define the name of each column and store it in a list (in the same order as our data). For this, we are going to create a new list named "cols".

In [18]:
cols = ['Date', 'Buy/Sell Operation', 'Stock Title', 'Price', 'Quantity']

Now that we have all the data stored in a list and our column names, we can create the data frame.

In [19]:
note_df = pd.DataFrame(data=note_data, columns=cols)
note_df

,Date,Buy/Sell Operation,Stock Title,Price,Quantity
0,08/03/2019,V,ENGIE BRASILON NM,"40,00",5
1,08/03/2019,C,GERDAUON ED N1,"11,63",10
2,08/03/2019,C,PARANAPANEMAON NM,"1,47",100


With the our data frame, we can call the method "to_excel" to export the it to a ".xlsx" file in our current directory with name "brokerage_note".

In [20]:
note_df.to_excel("brokerage_note.xlsx")